<a href="https://colab.research.google.com/github/eedduu/TFG/blob/main/code/old_playgrounds/TFG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prerrequisitos para usar FastAI
#!pip install -Uqq fastbook
!pip install nbdev
!pip install fastai
#import fastbook
#fastbook.setup_book()
import fastai
import fastcore
print(f'fastcore version {fastcore.__version__} installed')
print(f'fastai version {fastai.__version__} installed')
from nbdev.showdoc import *
from fastai.vision.all import *
import torch.nn.functional as F
import torchvision.transforms.functional as FF
import numpy
import torchvision
from torchvision import *

from sklearn.model_selection import StratifiedKFold

#!pip install google
#from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.w

In [ ]:

torch.manual_seed(42)
numpy.random.seed(42)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [ ]:
# Función para printar las gráficas de entrenamiento
# Developed by Ignacio Oguiza: https://forums.fast.ai/t/plotting-metrics-after-learning/69937

@patch
@delegates(subplots)
def plot_metrics(self: Recorder, nrows=None, ncols=None, figsize=None, **kwargs):
    metrics = np.stack(self.values)
    names = self.metric_names[1:-1]
    n = len(names) - 1
    if nrows is None and ncols is None:
        nrows = int(math.sqrt(n))
        ncols = int(np.ceil(n / nrows))
    elif nrows is None: nrows = int(np.ceil(n / ncols))
    elif ncols is None: ncols = int(np.ceil(n / nrows))
    figsize = figsize or (ncols * 6, nrows * 4)
    fig, axs = subplots(nrows, ncols, figsize=figsize, **kwargs)
    axs = [ax if i < n else ax.set_axis_off() for i, ax in enumerate(axs.flatten())][:n]
    for i, (name, ax) in enumerate(zip(names, [axs[0]] + axs)):
        ax.plot(metrics[:, i], color='#1f77b4' if i == 0 else '#ff7f0e', label='valid' if i > 0 else 'train')
        ax.set_title(name if i > 1 else 'losses')
        ax.legend(loc='best')
    plt.show()

In [ ]:
def LeNet5():
    return nn.Sequential(
    nn.Conv2d(1,6,kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(6,16,kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(400,120),
    nn.ReLU(),
    nn.Linear(120,84),
    nn.ReLU()
)

In [ ]:
#Cargo Datasets
path_mnist = untar_data(URLs.MNIST)
path_cifar = untar_data(URLs.CIFAR)
path_cifar100 = untar_data(URLs.CIFAR_100)
path_imagenette = untar_data(URLs.IMAGENETTE)

KeyboardInterrupt: 

In [ ]:
#Defino las particiones a usar


## MNIST
train_imgs= get_image_files(path_mnist/'training')
test_imgs= get_image_files(path_mnist/'testing')
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
item_tfms = [ToTensor(), Resize(32)]
batch_tfms = [IntToFloatTensor()]
dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImageBW.create], [parent_label, Categorize]])
dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)

train_labels=L(dls.train_ds.items).map(dls.tfms[1])
mnist=[]
for _, val_idx in kf.split(np.array(train_imgs+test_imgs), train_labels):

    splits = IndexSplitter(val_idx)
    split = splits(train_imgs+test_imgs)
    split_list = [split[0], split[1]]
    dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImageBW.create], [parent_label, Categorize]],
                    splits=split_list)
    dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)
    mnist.append(dls)


##CIFAR

train_imgs= get_image_files(path_cifar/'train')
test_imgs= get_image_files(path_cifar/'test')
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
item_tfms = [ToTensor(), Resize(64)]
batch_tfms = [IntToFloatTensor()]
dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]])
dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)

train_labels=L(dls.train_ds.items).map(dls.tfms[1])
cifar=[]
for _, val_idx in kf.split(np.array(train_imgs+test_imgs), train_labels):

    splits = IndexSplitter(val_idx)
    split = splits(train_imgs+test_imgs)
    split_list = [split[0], split[1]]
    dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]],
                    splits=split_list)
    dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)
    cifar.append(dls)


##CIFAR100
train_imgs= get_image_files(path_cifar100/'train')
test_imgs= get_image_files(path_cifar100/'test')
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
item_tfms = [ToTensor(), Resize(64)]
batch_tfms = [IntToFloatTensor()]
dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]])
dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)

train_labels=L(dls.train_ds.items).map(dls.tfms[1])
cifar100=[]
for _, val_idx in kf.split(np.array(train_imgs+test_imgs), train_labels):

    splits = IndexSplitter(val_idx)
    split = splits(train_imgs+test_imgs)
    split_list = [split[0], split[1]]
    dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]],
                    splits=split_list)
    dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)
    cifar100.append(dls)


##Imagenette
train_imgs= get_image_files(path_imagenette/'train')
test_imgs= get_image_files(path_imagenette/'val')
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
item_tfms = [ToTensor(), Resize(160)]
batch_tfms = [IntToFloatTensor()]
dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]])
dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)

train_labels=L(dls.train_ds.items).map(dls.tfms[1])
imagenette=[]
for _, val_idx in kf.split(np.array(train_imgs+test_imgs), train_labels):

    splits = IndexSplitter(val_idx)
    split = splits(train_imgs+test_imgs)
    split_list = [split[0], split[1]]
    dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImage.create], [parent_label, Categorize]],
                    splits=split_list)
    dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)
    imagenette.append(dls)





In [ ]:
torch.save(mnist, 'mnist.pth')
torch.save(cifar, 'cifar.pth')
torch.save(cifar100, 'cifar100.pth')
torch.save(imagenette, 'imagenette.pth')

model=LeNet5()
torch.save(model, 'LN5.pth')

model=alexnet()
torch.save(model, 'AN.pth')

model=resnet18()
torch.save(model, 'RN18.pth')

In [ ]:
for fold in mnist:
  model=torch.load('LN5.pth')
  learn=Learner(fold,model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, opt_func=SGD)

  learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,1.954529,1.884658,0.623254,01:10


epoch,train_loss,valid_loss,accuracy,time
0,2.537805,2.532074,0.453349,01:07


epoch,train_loss,valid_loss,accuracy,time
0,2.554210,2.535521,0.469164,01:03


In [ ]:
for fold in mnist:
  model=torch.load('LN5.pth')
  learn=Learner(fold,model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, opt_func=SGD)

  learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,2.536712,2.532062,0.508186,01:10


epoch,train_loss,valid_loss,accuracy,time
0,2.560414,2.535525,0.485621,00:58


epoch,train_loss,valid_loss,accuracy,time
0,1.968357,1.911709,0.554794,01:01


In [ ]:
#Dataset es un array con los distintos folds
def entrenar(dataset, model, opt, params=None, names=None, lr=0.001):
  if(opt=='SGD'):
    op=partial(OptimWrapper, torch.optim.SGD, lr=lr)
  elif(opt=='NAG'):
    op=partial(OptimWrapper, torch.optim.SGD, nesterov=True, momentum=params[0], lr=lr)
  elif(opt=='RMSProp'):
    op=partial(OptimWrapper, torch.optim.RMSProp, alpha=params[0], lr=lr)
  elif(opt=='AdamW'):
    op=partial(OptimWrapper, torch.optim.AdamW, betas=(params[0], params[1]), lr=lr)

  i=1
  for fold in dataset:
    learn=vision_learner(fold, alexnet, pretrained=False, loss_func=CrossEntropyLossFlat(), opt_func=SGD, metrics=accuracy, cbs=[SaveModelCallback(), EarlyStoppingCallback(0.01, 3)] )
    learn.fit(1)
    i+=1



In [ ]:
dls=torch.load('cifar.pth')
model=torch.load('AN.pth')
entrenar(dls, model, 'SGD')

In [ ]:
from numpy import random

train_imgs= get_image_files(path_mnist/'training')
test_imgs= get_image_files(path_mnist/'testing')
item_tfms = [ToTensor(), Resize(32)]
batch_tfms = [IntToFloatTensor()]

start_val= len(train_imgs+test_imgs) - len(train_imgs)
idxs=list(range(60000,70000))
splitter=IndexSplitter(idxs)
splits=splitter(train_imgs+test_imgs)

dsrc = Datasets(train_imgs+test_imgs, tfms=[[PILImageBW.create], [parent_label, Categorize]], splits=splits)
dls = dsrc.dataloaders(bs=32, after_item=item_tfms, after_batch=batch_tfms)

num_cros=5
num_elim=2
num_nuevos=2
modelos=[]
for i in range(num_cros):
  modelos.append(Learner(dls, LeNet5(), loss_func=CrossEntropyLossFlat(),metrics=accuracy))

test = dls.test_dl(test_imgs, with_labels=True)


In [ ]:
for model in modelos:
  model.fit(1, cbs=SaveModelCallback())

epoch,train_loss,valid_loss,accuracy,time
0,1.826931,1.771857,0.605100,01:47


Better model found at epoch 0 with valid_loss value: 1.7718573808670044.


epoch,train_loss,valid_loss,accuracy,time
0,0.624150,0.570558,0.867000,01:49


Better model found at epoch 0 with valid_loss value: 0.5705575346946716.


epoch,train_loss,valid_loss,accuracy,time
0,0.956134,0.951353,0.788100,01:40


Better model found at epoch 0 with valid_loss value: 0.9513528347015381.


epoch,train_loss,valid_loss,accuracy,time
0,2.850323,2.772205,0.456900,01:38


Better model found at epoch 0 with valid_loss value: 2.772204875946045.


epoch,train_loss,valid_loss,accuracy,time
0,1.006269,0.962110,0.862100,01:39


Better model found at epoch 0 with valid_loss value: 0.9621097445487976.


In [ ]:
fitness=[]
for model in modelos:
  loss=model.recorder.values[-1][1]
  fitness.append(np.array(loss).mean())
indices = np.argsort([fitness])

In [ ]:
a=modelos_ord[eleg1].model.state_dict()
b=modelos_ord[eleg2].model.state_dict()
dic1=OrderedDict()
dic2=OrderedDict()


for key in a.keys():
  prob1 = random.uniform(0,1)
  prob2 = random.uniform(0,1)
  dic1[key]=prob1*(b[key]+0.5*(abs(a[key]-b[key]) - a[key]-0.5*(abs(a[key]-b[key])))) + a[key]-0.5*(abs(a[key]-b[key]))
  dic2[key]=prob2*(b[key]+0.5*(abs(a[key]-b[key]) - a[key]-0.5*(abs(a[key]-b[key])))) + a[key]-0.5*(abs(a[key]-b[key]))

  #for (elem1,elem2) in zip(a[key],b[key]):
  #  val1=random.uniform(elem1-0.5*(abs(elem1-elem2)), ))



In [ ]:
modelos_orig=modelos


In [ ]:
fitness_orig=[]
modelos=modelos_orig
for model in modelos:
  loss=model.get_preds(dl=test, with_loss=True)[2]  #model.recorder.values[-1][1]
  fitness_orig.append(np.array(loss).mean())
fitness=np.array(fitness_orig)
indices = np.argsort([fitness])

aux1 = Learner(dls, LeNet5(), loss_func=CrossEntropyLossFlat(),metrics=accuracy)
aux2 = Learner(dls, LeNet5(), loss_func=CrossEntropyLossFlat(),metrics=accuracy)


indices = np.argsort([fitness])
print("Orden de mejores hijos: {}".format(indices))


modelos_ord = np.array(modelos)[indices[0].astype(int)]
fitness=fitness[indices[0].astype(int)]

print(fitness)
for j in range(200):
  print("EPOC NUMERO {}".format(j+1))





  aleat1= random.randint(0,4)
  aleat2= random.randint(0,4)
  aleat3= random.randint(0,4)

  if(fitness[aleat1]<=fitness[aleat2] and fitness[aleat1]<=fitness[aleat3]):
    eleg1=aleat1
  elif(fitness[aleat2]<=fitness[aleat1] and fitness[aleat2]<=fitness[aleat3]):
    eleg1=aleat2
  else:
    eleg1=aleat3

  aleat1= random.randint(0,4)
  aleat2= random.randint(0,4)
  aleat3= random.randint(0,4)

  if(fitness[aleat1]<=fitness[aleat2] and fitness[aleat1]<=fitness[aleat3]):
    eleg2=aleat1
  elif(fitness[aleat2]<=fitness[aleat1] and fitness[aleat2]<=fitness[aleat3]):
    eleg2=aleat2
  else:
    eleg2=aleat3

  a=modelos_ord[eleg1].model.state_dict()
  b=modelos_ord[eleg2].model.state_dict()
  dic1=OrderedDict()
  dic2=OrderedDict()

  for key in a.keys():
    prob1 = random.uniform(0,1)
    prob2 = random.uniform(0,1)
    dic1[key]=prob1*(b[key]+0.5*(abs(a[key]-b[key]) - a[key]-0.5*(abs(a[key]-b[key])))) + a[key]-0.5*(abs(a[key]-b[key]))
    dic2[key]=prob2*(b[key]+0.5*(abs(a[key]-b[key]) - a[key]-0.5*(abs(a[key]-b[key])))) + a[key]-0.5*(abs(a[key]-b[key]))

  aux1.model.load_state_dict(dic1)
  aux2.model.load_state_dict(dic2)

  _, _, loss= aux1.get_preds(dl=test, with_loss=True)
  modelos_ord=np.append(modelos_ord, aux1)
  fitness=np.append(fitness, np.array(loss).mean())

  _, _, loss= aux2.get_preds(dl=test, with_loss=True)
  modelos_ord=np.append(modelos_ord, aux2)
  fitness=np.append(fitness, np.array(loss).mean())

  indices=np.argsort([fitness])
  fitness=fitness[indices[0].astype(int)]

  print(fitness)

  modelos= modelos_ord[indices[0].astype(int)]
  modelos= modelos[0:5]
  fitness=fitness[0:5]

  print(fitness)

  if (j%20==0):
    for model in modelos:
      model.fit(1, cbs=SaveModelCallback())





/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Orden de mejores hijos: [[3 2 0 1 4]]
[4.407029  4.425091  4.4318585 4.4322815 4.440098 ]
EPOC NUMERO 1


[4.407029  4.42463   4.425091  4.426381  4.4318585 4.4322815 4.440098 ]
[4.407029  4.42463   4.425091  4.426381  4.4318585]


epoch,train_loss,valid_loss,accuracy,time
0,0.068342,0.070217,0.976300,01:40


Better model found at epoch 0 with valid_loss value: 0.07021667808294296.


...

epoch,train_loss,valid_loss,accuracy,time
0,0.951740,0.893565,0.799700,01:42


Better model found at epoch 0 with valid_loss value: 0.8935654163360596.


epoch,train_loss,valid_loss,accuracy,time
0,0.971405,0.931676,0.791400,01:41


Better model found at epoch 0 with valid_loss value: 0.9316759705543518.


epoch,train_loss,valid_loss,accuracy,time


In [ ]:
fitness_orig

[0.5109903812408447,
 0.9078168869018555,
 0.48848453164100647,
 0.9637036919593811,
 1.0294127464294434]